### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import sklearn
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import scipy
from datasets import load_dataset

General Workflow:




1. Speech signal ---> Text




2. Text ----> Dirichlet Clusters




3. Dirichlet Cluster keywords ---> Search Engine / Youtube API







In [2]:
### Loading model to device

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-medium.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])

config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

c:\Users\yangz\anaconda3\envs\procrastinate\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yangz\.cache\huggingface\hub\models--openai--whisper-medium.en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

In [ ]:
sample = scipy.io.wavfile.read("alejandro_voice.wav")
voice_sample = sample[1]
left_channel_voice = voice_sample[:, 0]
right_channel_voice = voice_sample[:, 1]
print(left_channel_voice)

### Try with left channel and compare with right channel compare the text differences

In [ ]:
left_result = pipe(left_channel_voice, generate_kwargs={"language": "english"})
print(left_result["text"])

In [ ]:
right_result = pipe(right_channel_voice, generate_kwargs={"language": "english"})
print(right_result["text"])